In [16]:
library(sp) #provides classes and methods for spatial data
library(rgdal) #provides binding to Geospatial Data Abstraction Library (GDAL)
library(rgeos) # provides interface to Geometry Engine - Open Source (GEOS)
library(ggmap)
# library(maptools)

Loading required package: ggplot2

Google's Terms of Service: https://cloud.google.com/maps-platform/terms/.

Please cite ggmap if you use it! See citation("ggmap") for details.



In [30]:
# read a shape file into an R object shape
# * download shape file .shp from:
# http://qldspatial.information.qld.gov.au/catalogue/custom/detail.page?fid={079E7EF8-30C5-4C1D-9ABF-3D196713694F}
#
electoral_boundaries_2017 <- readOGR("2017 electorate boundaries/QSC_Extracted_Data_20200930_142910272000-85432/State_electoral_boundaries_2017.shp")

Warning message in OGRSpatialRef(dsn, layer, morphFromESRI = morphFromESRI, dumpSRS = dumpSRS, :
“Discarded datum Geocentric_Datum_of_Australia_1994 in CRS definition: +proj=longlat +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +no_defs”


OGR data source with driver: ESRI Shapefile 
Source: "/Volumes/apv/CJRS/uni/03 UQ MDataSc/04 S2 2020/DATA7001/9 Group Project/infrastructure_by_electorate/2017 electorate boundaries/QSC_Extracted_Data_20200930_142910272000-85432/State_electoral_boundaries_2017.shp", layer: "State_electoral_boundaries_2017"
with 93 features
It has 4 fields


ERROR: Error in spTransform(electoral_boundaries_2017, CRSobj = NA): second argument needs to be of class CRS


In [19]:
# register Google API key
# (To get this one needs to register for a billing account with Google Maps Platform.)
register_google(key = "[redacted]")

In [75]:
# Demonstration: determine whether a point is in an electorate.
# Note that SuppressWarnings() is necessary to suppress lengthy warnings about inconsistent projection/coordinate system
# between first and second argument to gContains(). This does not seem to point to any real problem in our case - we have
# checked and confirmed that the comparison is working correctly.

# is Brisbane Airport in electoral district of Noosa?
suppressWarnings(gContains(shape[electoral_boundaries_2017@data$NAME=="NOOSA",], SpatialPoints(geocode("Brisbane Airport"))))

# is the Big Peanut in electoral district of Everton?
suppressWarnings(gContains(shape[electoral_boundaries_2017@data$NAME=="EVERTON",], SpatialPoints(geocode("The Big Peanut, Kingaroy"))))

# is UQ in electoral district of Maiwar?
suppressWarnings(gContains(shape[electoral_boundaries_2017@data$NAME=="MAIWAR",], SpatialPoints(geocode("University of Queensland"))))

gContains(shape[electoral_boundaries_2017@data$NAME=="MAIWAR",], SpatialPoints(geocode("University of Queensland")))

Source : https://maps.googleapis.com/maps/api/geocode/json?address=Brisbane+Airport&key=xxx



[1] FALSE

Source : https://maps.googleapis.com/maps/api/geocode/json?address=The+Big+Peanut,+Kingaroy&key=xxx



[1] FALSE

Source : https://maps.googleapis.com/maps/api/geocode/json?address=University+of+Queensland&key=xxx



[1] TRUE

Source : https://maps.googleapis.com/maps/api/geocode/json?address=University+of+Queensland&key=xxx

Warning message in RGEOSBinPredFunc(spgeom1, spgeom2, byid, func):
“spgeom1 and spgeom2 have different proj4 strings”


[1] TRUE

In [57]:
# We can extend this idea to take a function which takes a spatialpoint and searches for it
# in each of the 93 electorates, and returns the name of the electorate in which it is
# found:

find_electorate <- function(spoint, electoral_boundaries) {
        found_electorate = NA
        for (electorate in electoral_boundaries@data$NAME) {
                if(suppressWarnings(gContains(shape[shape@data$NAME==electorate,], spoint))) {
                        found_electorate <- electorate
                }
        }
        return(found_electorate)
}

In [74]:
# Demonstration:

find_electorate(SpatialPoints(geocode("University of Queensland")), electoral_boundaries_2017)
find_electorate(SpatialPoints(geocode("Breakfast Creek Hotel, Albion")), electoral_boundaries_2017)

Source : https://maps.googleapis.com/maps/api/geocode/json?address=University+of+Queensland&key=xxx



[1] "MAIWAR"

Source : https://maps.googleapis.com/maps/api/geocode/json?address=Breakfast+Creek+Hotel,+Albion&key=xxx



[1] "CLAYFIELD"

In [ ]:
find_electorate_by_latlong <- function(latitude, longitude, electorates) {
        spoint <- SpatialPoints(tibble(lon=longitude, lat=latitude))
        return(find_electorate(spoint, electorates))
}